# Adding custom metadata
It is possible to attach and store arbitrary metadata to WelDX objects as python attributes.
The WelDX API reserves the attribute names `wx_metadata` and `wx_user` for these use cases.

The `wx_metadata` field is intended for additional information that might be required as part of a custom Quality Standard. The `wx_user` attribute is intended to add any user data and should only be used for additional information that will not get accessed by the API. 

## A small example
Let's say we want to use metadata to add a custom information to a current-sensor used during a welding experiment. As an example we want to add a timestamp to indicate when the sensor was last calibrated.

In [ ]:
# if the package is not installed in your python environment, run this to execute the notebook directly from inside the GitHub repository
%cd -q ..

We start by by creating the sensor object as a `GenericEquipment`:

In [ ]:
from weldx.measurement import GenericEquipment

HKS_sensor = GenericEquipment(name="HKS P1000-S3")

Now we want to add the timestamp of the last time the sensor was calibrated as additional metadata. To do this we simple assign a new `.wx_metadata` attribute to the sensor. We will also add a small personal note to `wx_user`.

*Note: By convention the metadata fields should be python dictionaries.*

In [ ]:
import pandas as pd

HKS_sensor.wx_metadata = {"calibration": pd.Timestamp("2020-06-01")}
HKS_sensor.wx_user = {"notes": ["wallmounted", "The cable seems to be a bit loose."]}

Now let's look at the ASDF file contents with our metadata:

In [ ]:
from weldx.asdf.utils import _write_buffer, notebook_fileprinter

buffer = _write_buffer({"sensor": HKS_sensor})
notebook_fileprinter(buffer)

As we can see, our custom metadata gets serialized into the ASDF tree:
```yaml
wx_metadata:
    calibration: !<tag:weldx.bam.de:weldx/time/timestamp-1.0.0> {value: '2020-06-01T00:00:00'}
  wx_user:
    notes: [wallmounted, The cable seems to be a bit loose.]
```
Let's check if we keep the information upon reading the file:

In [ ]:
from weldx.asdf.utils import _read_buffer

data = _read_buffer(buffer)
display(data["sensor"].wx_metadata)
display(data["sensor"].wx_user)